In [ ]:
# File: 01_get_article_data
# Author: Steven Pryor, github.com/troub1
# Retrieves and/or processes article data from .csv file
    #currently (Feb 2021), this has been done by searching the Scopus web interface and downloading a CSV. The .csv is then imported here as a dataframe for processing
# Retrieves and combines additional data from Unpaywall

# 1. Download raw data from Scopus (and/or WoS, though check field names)
# 2. Clean up with OpenRefine
#    a. Publisher names
#    b. 
#    c. 
# 3. Read cleaned file, get additional OA data from Unpaywall for each DOI
# 4. Get contact info, names, and department for each MU Corresponding Author (store a file/database of already-collected authors to check first before LDAP lookup? This could also help w/ future process for non-corresponding authors)
# 5. Match librarians to each author by department
# 6. Gold OA articles
#    a. Send emails to authors and librarians notifying of MOspace collection of OA articles
#    b. Retrieve articles
#    c. Create metadata and batch ingest files
# 7. CREATE PROCESS HERE FOR Green-Available (also harvest? Could be issues with other repos' cover pages, copyright statements, licenses...other things?)
# 8. CREATE PROCESS HERE FOR Green-Potential
#        - Need to figure out how to best identify the authors
# 9. CREATE PROCESS FOR UPDATE
#      - Check for new/updated Unpaywall data?
#      - Check for new articles or new/updated Scopus data
#      - Add and combine with WoS or other data sources to expand coverage

In [ ]:
import json, re
import pandas as pd
import requests, time

test_doi = "10.1080/01930826.2014.893110"
api_unpaywall = "https://api.unpaywall.org/v2/"
params_unpaywall = {'email':''} #enter your email address here
unpaywall_results = []

infile = '' # input: filename for raw Scopus data (or whatever data file with DOIs to be augmented with Unpaywall data)
outfile = '' # output: filename for Scopus data with added Unpaywall columns

#---------------
#Choose one of the functions below based on whether your input file is .csv or .xls
#scopus_Data = pd.read_excel(infile)
#scopus_Data = pd.read_csv(infile, encoding='ANSI')


In [ ]:
##This function loops through the DOI list and queries unpaywall for any known OA locations for each DOI. The result is a json object identifying whether each document has an OA location, the best OA location (if any)
##URLs to pdfs, etc. The json results will be saved into a list (unpaywall_results) as well as written to a file (50,000+ API calls takes several hours to complete; save it when you're done! Could multithread, bah!)

def get_unpaywall(doi_list):
    for doi in doi_list:
        if not pd.isnull(doi):
            try:
                result = requests.get(api_unpaywall + doi, params=params_unpaywall)
            except (requests.HTTPError, requests.RequestException) as e:
                print("Error: " + e)
            unpaywall_results.append(result.json())
        else:
            unpaywall_results.append({})
        
        print(".", end="")
        if not len(unpaywall_results)%5000:
            print("\n(+5K) Still working...")
    print("Done\n")
    
    ##Now read in the unpaywall results to a pandas DataFrame so we can play around with it
    unpaywall_Data = pd.read_json(path_or_buf=json.JSONEncoder().encode(unpaywall_results), orient="records")
    
    print("COMPARISON:\n")
    print(unpaywall_Data['title'][:-9], '\n===================\n')
    print(scopus_Data['Title'][:-9])
    
    ##The unpaywall json result contains lists as elements, ie the best_oa_location 'column' of the DataFrame contains a list of the location URL, license, etc. We need to "flatten" or expand this list out, so we'll
    ##extract it as a series, rename the columns with a prefix indicating the original location, and then concatenate the columns onto the DataFrame/table. This lets us see and reference the list nested inside
    ##the best_oa_location column and perform operations/counts/etc. on it
    best_oa = unpaywall_Data['best_oa_location'].apply(pd.Series)
    best_oa = best_oa.rename(columns = lambda x : 'best_oa_' + str(x))
    ##Also expand out the list of authors
    #authors = df['z_authors'].apply(pandas.Series)
    #authors = authors.rename(columns = lambda x : 'authors' + str(x))
    unpaywall_Data_expanded = pd.concat([unpaywall_Data[:], best_oa[:]], axis=1)
    
    return unpaywall_Data_expanded

In [ ]:
unpaywall_df = get_unpaywall(scopus_Data['DOI'])

In [ ]:
#And now combine the Unpaywall data we want with the rest of the data from Scopus by adding columns:
scopus_Data['best_oa_license'] = unpaywall_df['best_oa_license']
scopus_Data['pubdate'] = unpaywall_df['published_date']
scopus_Data['oa_status'] = unpaywall_df['oa_status']
scopus_Data['best_oa_evidence'] = unpaywall_df['best_oa_evidence']
scopus_Data['best_oa_url_for_pdf'] = unpaywall_df['best_oa_url_for_pdf']
scopus_Data['best_oa_url'] = unpaywall_df['best_oa_url']
scopus_Data['best_oa_version'] = unpaywall_df['best_oa_version']
scopus_Data['journal_is_oa'] = unpaywall_df['journal_is_oa']
scopus_Data['is_oa'] = unpaywall_df['is_oa']

In [ ]:
# Add a "green open access" potential OA column
# Use list of defined preferred publisher names (for OpenRefine step)
# Build list of Publishers with confirmed Green OA policies of some kind
# These are matches from high-number publishers in one of our data sets; if your faculty publish different places you might want to check different publishers and add some here, too
green_oa_pubs = ['Elsevier Ltd', 'Blackwell Publishing Ltd', 'John Wiley and Sons Inc.', 'Springer', 'Nature Publishing Group', 'Springer International Publishing', 'Informa UK Limited', 'SAGE Publications Inc.', 
                 'American Chemical Society', 'Elsevier', 'Public Library of Science (PLoS)', 'Frontiers Media SA', 'Oxford University Press', 'Ovid Technologies (Wolters Kluwer Health)', 
                 'IEEE', 'Institute of Electrical and Electronics Engineers Inc.', 'American Physical Society (APS)', 'Taylor and Francis', 'Routledge' ]
oa_potential = []

for index, row in scopus_Data.iterrows():
    # For "potential OA" count, count whether the publisher has a green OA policy *or* it's already available (we want to add up the total of already-OA and could-be-OA)
    if( (row['Publisher'] in green_oa_pubs) or row['is_oa'] or row['journal_is_oa'] ):
       oa_potential.append('1')
    else:
       oa_potential.append('0')

In [ ]:
scopus_Data['potential_oa'] = oa_potential

In [ ]:
#Save our work!
scopus_Data.to_csv(outfile, encoding='utf-8-sig')

In [ ]:
#===================================================================================
#END data collection code